## Exercise on apriori

In this exercise, you have to create the Apriori algorithm.

In [1]:
from itertools import combinations,chain

Write a function to find all combination of itemsets of size level-1 to generate new level-size itemsets.

In [2]:
def mingle(items, level):
    
    outcome = set()
    for item in items:
        for item2 in items:
            if item!=item2:
                newCombination = set()
                # If level >2, this means the itemsets contain more than 1 item
                if level>2:
                    for i in item:
                        newCombination.add(i)
                    for i in item2:
                        newCombination.add(i)
                else:
                    newCombination.add(item)
                    newCombination.add(item2)
                # Only retain combinations of itemsets that are of the size of the current level
                # The combination of larger itemsets can result in, e.g., 2-item itemsets combined 
                #                 into 4-item itemsets
                # while the level is 3, requiring 3-item itemsets
                if(len(newCombination)==level):
                    outcome.add(frozenset(newCombination))
                    
    return outcome

assert  mingle(["a","b","c"], 2) == {frozenset({'a', 'c'}), 
                                     frozenset({'b', 'c'}), 
                                     frozenset({'a', 'b'})}

assert mingle([["a","b"],["a","c"],["a","d"]], 3) == {frozenset({'a', 'c', 'd'}), 
                                               frozenset({'a', 'b', 'd'}),
                                               frozenset({'a', 'b', 'c'})}

Write a function that calculates the support of an itemset in a transactions database.

In [3]:
def support(itemset,transactions,level):
    
    count = 0
    for trans in transactions:
        # Assume the transaction contains the items unless proven otherwise below
        contain = True
        # If level > 1, the itemsets contain more than 1 item, and we need to loop all items in the itemset
        if level>1:
            for item in itemset:      
                if item not in trans:
                    # No need to look further if even 1 item is not contained in the transaction
                    contain = False
                    break
        else:
            if itemset not in trans:
                contain = False
        if contain:
            count = count + 1
    return count/len(transactions)

assert support("a", [["a","b","c"], ["a","b","d"], ["b","c"], ["a","c"]], 1) == 0.75
assert support("d", [["a","b","c"], ["a","b","d"], ["b","c"], ["a","c"]], 1) == 0.25
assert support(["a","b"], [["a","b","c"], ["a","b","d"], ["b","c"], ["a","c"]], 2) == 0.5



Write the Apriori function.

In [4]:
# for now this function will just print some results for us to observe, 
# rather than return them in a data structure

def apriori(level,transactions,items,minsup):
    
    print("\nLevel: "+str(level))
    
    # set for items with support value that is high enough
    retain = set()
    
    # find items with support value that is high enough
    for item in items:
        print(str(item)+" support: "+str(support(item,transactions,level)))      
        if support(item,transactions,level)>=minsup:
            retain.add(item)
    print("Retain: "+str(retain))
    
    level = level+1
        
    # generate new candidates
    newsets=mingle(retain,level)
    print("New itemsets: "+str(newsets))    
    
    # stop if no candidates are left or you will put all items in one set
    if len(newsets)!=0 and level<len(items)+1:
        apriori(level,transactions,newsets,minsup)
        
apriori(1, [["a","b","c"], ["a","b","d"], ["b","c"]], {"a","b","c", "d"}, 0.6)


Level: 1
c support: 0.6666666666666666
b support: 1.0
a support: 0.6666666666666666
d support: 0.3333333333333333
Retain: {'b', 'c', 'a'}
New itemsets: {frozenset({'b', 'a'}), frozenset({'b', 'c'}), frozenset({'c', 'a'})}

Level: 2
frozenset({'b', 'a'}) support: 0.6666666666666666
frozenset({'b', 'c'}) support: 0.6666666666666666
frozenset({'c', 'a'}) support: 0.3333333333333333
Retain: {frozenset({'b', 'a'}), frozenset({'b', 'c'})}
New itemsets: {frozenset({'c', 'b', 'a'})}

Level: 3
frozenset({'c', 'b', 'a'}) support: 0.3333333333333333
Retain: set()
New itemsets: set()


Use this to run the complete algorithm.

In [59]:
# open the data
file = open('baskets.csv','r')

transactions = []
items = set()

# save all transactions and items
for line in file:
    line = line.replace('\n','')
    litems = line.split(',')
    transactions.append(litems)
    for item in litems:
        items.add(item)

noItems = len(items)

# apply Apriori algorithm
apriori(1,transactions,items,0.6)   

O 1 [['iphoneX', 'S8', 'LG55', 'S9'], ['iphoneX', 'S8', 'S9', 'LG55'], ['LG55', 'S2', 'iphoneX'], ['iphoneX', 'S8', 'LG55'], ['S8', 'S9']] {'S8', 'LG55', 'S9', 'S2', 'iphoneX'} 0.6

Level: 1
S8 support: 0.8
LG55 support: 0.8
S9 support: 0.6
S2 support: 0.2
iphoneX support: 0.8
Retain: {'S9', 'S8', 'iphoneX', 'LG55'}
New itemsets: {frozenset({'S8', 'LG55'}), frozenset({'S9', 'S8'}), frozenset({'S8', 'iphoneX'}), frozenset({'S9', 'LG55'}), frozenset({'iphoneX', 'LG55'}), frozenset({'S9', 'iphoneX'})}
O 2 [['iphoneX', 'S8', 'LG55', 'S9'], ['iphoneX', 'S8', 'S9', 'LG55'], ['LG55', 'S2', 'iphoneX'], ['iphoneX', 'S8', 'LG55'], ['S8', 'S9']] {frozenset({'S8', 'LG55'}), frozenset({'S9', 'S8'}), frozenset({'S8', 'iphoneX'}), frozenset({'S9', 'LG55'}), frozenset({'iphoneX', 'LG55'}), frozenset({'S9', 'iphoneX'})} 0.6

Level: 2
frozenset({'S8', 'LG55'}) support: 0.6
frozenset({'S9', 'S8'}) support: 0.6
frozenset({'S8', 'iphoneX'}) support: 0.6
frozenset({'S9', 'LG55'}) support: 0.4
frozenset({'ip